# create CMOR demo defining grid mapping

## Notes

**Summary**

Quick and dirty notebook to load data from a demo file, and write to CMOR complete with defined grid mapping

**Authors**

Paul J. Durack ([durack1](https://github.com/durack1); [PCMDI](https://pcmdi.llnl.gov/), [Lawrence Livermore National Laboratory](https://www.llnl.gov/))

**Notes**

PJD 29 Jan 2025 - Started<br>

**Links**

### imports

In [1]:
%%time
import cftime
import cmor
import json
import os
import numpy as np
import xarray as xr
import xcdat as xc

CPU times: user 552 ms, sys: 267 ms, total: 818 ms
Wall time: 1.36 s


### where am i?

In [2]:
print(os.getcwd())

/Users/durack1/sync/git/notebooks/jlnbs


### load demo data

In [3]:
filePath = "/Users/durack1/Downloads"
fileName = "macav3_ACCESS-CM2_historical_2009.nc"
filePathAndName = os.path.join(filePath, fileName)
print(filePathAndName)

/Users/durack1/Downloads/macav3_ACCESS-CM2_historical_2009.nc


In [4]:
fH = xc.open_dataset(filePathAndName)
fH

<xarray.Dataset> Size: 25GB
Dimensions:    (crs: 1, time: 365, lat: 671, lon: 1271, bnds: 2)
Coordinates:
  * crs        (crs) int16 2B 3
  * lat        (lat) float64 5kB 7.5 7.6 7.7 7.8 7.9 ... 74.2 74.3 74.4 74.5
  * lon        (lon) float64 10kB -179.5 -179.4 -179.3 ... -52.7 -52.6 -52.5
  * time       (time) object 3kB 2009-01-01 00:00:00 ... 2009-12-31 00:00:00
Dimensions without coordinates: bnds
Data variables: (12/13)
    hursmax    (time, lat, lon) float64 2GB ...
    hursmin    (time, lat, lon) float64 2GB ...
    pr         (time, lat, lon) float64 2GB ...
    rsds       (time, lat, lon) float64 2GB ...
    tasmax     (time, lat, lon) float64 2GB ...
    tasmin     (time, lat, lon) float64 2GB ...
    ...         ...
    uas        (time, lat, lon) float64 2GB ...
    vas        (time, lat, lon) float64 2GB ...
    was        (time, lat, lon) float64 2GB ...
    lon_bnds   (lon, bnds) float64 20kB -179.6 -179.4 -179.4 ... -52.55 -52.45
    lat_bnds   (lat, bnds) float64 11kB 7.45 7.55 7.55 ... 74.45 74.45 74.55
    time_bnds  (time, bnds) object 6kB 2008-12-31 12:00:00 ... 2009-12-31 12:...
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-179.5 74.5,-179.5 7.5,-52.5 7.5,-52...
    geospatial_lat_min:         7.5
    geospatial_lat_max:         74.5
    geospatial_lon_min:         -179.5
    ...                         ...
    date:                       03 May 2024
    note1:                      The projection information for this file is: ...
    note2:                      These data were created using netCDF version 4.
    description:                Draft of MACAv3 statistical downscaling of da...
    history:                    Sun May  5 12:25:51 2024: ncks --mk_rec_dmn t...
    NCO:                        netCDF Operators version 4.8.1 (Homepage = ht...

### create single variable with two timesteps

In [5]:
pr = fH.pr[0:2,:,:].data
pr = fH.pr.sel(time=slice("2008-12-31", "2009-01-02"))
time_bnds = fH.time_bnds.sel(time=slice("2008-12-31", "2009-01-02"))
time_bnds = cftime.date2num(time_bnds, fH.time.description)
#type(time_bnds[0,0])
time_bnds = time_bnds.astype("float32")
#time_bnds
time = cftime.date2num(pr.time, fH.time.description)
time = time.astype("float32")
#type(time[0])
#time
pr

<xarray.DataArray 'pr' (time: 2, lat: 671, lon: 1271)> Size: 14MB
[1705682 values with dtype=float64]
Coordinates:
  * lat      (lat) float64 5kB 7.5 7.6 7.7 7.8 7.9 ... 74.1 74.2 74.3 74.4 74.5
  * lon      (lon) float64 10kB -179.5 -179.4 -179.3 ... -52.7 -52.6 -52.5
  * time     (time) object 16B 2009-01-01 00:00:00 2009-01-02 00:00:00
Attributes:
    units:              mm
    description:        Daily Total Precipitation
    long_name:          precipitation
    standard_name:      precipitation
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326

### get crs variable attributes as dictionary

In [6]:
crsParams = fH.crs.attrs
crsParams

{'grid_mapping_name': 'latitude_longitude',
 'longitude_of_prime_meridian': np.float64(0.0),
 'semi_major_axis': np.float64(6378137.0),
 'long_name': 'WGS 84',
 'inverse_flattening': np.float64(298.257223563),
 'GeoTransform': '-179.5 0.1 0 74.5 0.1',
 'spatial_ref': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'}

### initialize cmor, load tables etc

In [7]:
# Start CMORising
cmor.setup(
    netcdf_file_action=cmor.CMOR_REPLACE_4,
)
# inpath="CMOR/input4MIPs-cmor-tables/Tables",
dummyUserInputFile = "CMAP-V1902-input.json"
cmor.dataset_json(dummyUserInputFile)
d = pr
lat = pr.lat.values
lon = pr.lon.values
#time = pr.time
# Force local file attribute as history
cmor.set_cur_dataset_attribute("history", "something useful")
table = "obs4MIPs_Aday.json"
tablePath = "/Users/durack1/sync/git/obs4MIPs-cmor-tables/Tables"
with open(os.path.join(tablePath, table)) as fh:
    tmp = json.load(fh)
# Fudge table files to force project=DRCDP
tmp["Header"]["mip_era"] = "DRCDP"
oH = open("tmp.json", "w")
json.dump(
    tmp,
    oH,
    ensure_ascii=True,
    sort_keys=True,
    indent=4,
    separators=(
        ',',
        ':')
)
oH.close()
cmor.load_table("tmp.json")
os.remove("tmp.json")

axes = [
    {
        "table_entry": "time",
        "units": "days since 1900-01-01",
        "coord_vals": time,
        "cell_bounds": time_bnds,
    },
    {
        "table_entry": "latitude",
        "units": "degrees_north",
        "coord_vals": lat,
        "cell_bounds": fH.lat_bnds[:],
    },
    {
        "table_entry": "longitude",
        "units": "degrees_east",
        "coord_vals": lon,
        "cell_bounds": fH.lon_bnds[:],
    },
]
axis_ids = list()
for axis in axes:
    axis_id = cmor.axis(**axis)
    axis_ids.append(axis_id)

# explicitly define grid axes
entry = {
    "table_entry": "longitude",
    "units": "degrees_east",
    "coord_vals": lon,
    "cell_bounds": fH.lon_bnds[:],
}
#axisX = cmor.axis(**entry)
entry = {
    "table_entry": "latitude",
    "units": "degrees_north",
    "coord_vals": lat,
    "cell_bounds": fH.lat_bnds[:],
}
#axisY = cmor.axis(**entry)

# grid mapping
# https://cmor.llnl.gov/mydoc_cmor3_api/#cmor_set_grid_mapping
grid_id = cmor.grid(
#            axis_ids=[
#                axisY,
#                axisX
#            ],
            axis_ids=axis_ids,
            time=time,
            latitude=lat,
            longitude=lon,
            )
ierr = cmor.set_grid_mapping(
            grid_id, "latitude_longitude", crsParams)
"""
print("varName:", "pr", "units:", pr.units, "axis_ids:", axis_ids)
#varid = cmor.variable("pr", "kg m-2 s-1", axis_ids)
#values = np.array(pr[:], np.float32)
# shuffle=1,deflate=1,deflate_level=1 ; CMOR 3.0.6+
#cmor.set_deflate(varid, 1, 1, 1)
#cmor.write(varid, values, time_vals=time, time_bnds=time_bnds)
"""
cmor.close()

TypeError: grid() got an unexpected keyword argument 'time'